<a href="https://colab.research.google.com/github/neeraj-pola/Time-Table-Generator/blob/main/Time_table_generator_using_pulp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install PuLP

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 22.7 MB/s eta 0:00:00


In [2]:
import pulp

In [3]:
# Define parameters
faculties = ["FacultyA", "FacultyB", "FacultyC", "FacultyD"]
subjects = ["Math", "Science", "History", "Telugu"]
classes = ["ClassX", "ClassY"]
days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday"]
periods_per_day = [1, 2, 3, 4, 5]

In [4]:
# Define a custom order for days
custom_day_order = {"Monday": 1, "Tuesday": 2, "Wednesday": 3, "Thursday": 4, "Friday": 5, "Saturday": 6}
sorted_days = sorted(days, key=lambda day: custom_day_order[day])


In [5]:
# Define a dictionary to specify faculty-subject qualifications
faculty_subject_qualifications = {
    "FacultyA": ["Math", "Science"],
    "FacultyB": ["Science", "History"],
    "FacultyC": ["Math", "History"],
    "FacultyD": ["Telugu"]
}

In [6]:
# Define a dictionary to specify which subjects each faculty teaches for each class
faculty_class_subjects = {
    "FacultyA": {"ClassX": ["Math"], "ClassY": ["Math"]},
    "FacultyB": {"ClassX": ["Science"], "ClassY": ["History"]},
    "FacultyC": {"ClassX": ["History"], "ClassY": ["Science"]},
    "FacultyD": {"ClassX": ["Telugu"], "ClassY": ["Telugu"]}
}

In [7]:
# Create a PuLP optimization problem
problem = pulp.LpProblem("Timetable_Generator", pulp.LpMinimize)

In [8]:
# Decision variables
x = pulp.LpVariable.dicts("Assign", ((i, j, k, day, period) for i in faculties for j in subjects for k in classes for day in sorted_days for period in periods_per_day), cat="Binary")


#Constraints

##Each subject should be taught by exactly one faculty during each period
###x[i,j,k,l] is a binary decision that indicates whether faculty i teaches subject j for class k in period l
###i -> faculty index
###j -> subject index
###k-> class index
###l -> period index
###summation(i,k) gives all the possible combinations of faculties and the classes for given subject j in l


In [9]:
# Constraint: Each subject is assigned to exactly one class during each period on each day
for j in subjects:
    for day in sorted_days:
        for period in periods_per_day:
            problem += pulp.lpSum(x[i, j, k, day, period] for i in faculties for k in classes) == 1


In [10]:
# Constraint: Faculties can only teach subjects they are qualified for
for i in faculties:
    for j in subjects:
        for k in classes:
            for day in sorted_days:
                for period in periods_per_day:
                    if j not in faculty_subject_qualifications[i]:
                        problem += x[i, j, k, day, period] == 0


In [11]:
# Constraint: Assign each subject for a class to a specific faculty based on faculty_class_subjects
for k in classes:
    for j in subjects:
        for day in sorted_days:
            for period in periods_per_day:
                assigned_faculty = None
                for faculty, class_subjects in faculty_class_subjects.items():
                    if k in class_subjects and j in class_subjects[k]:
                        assigned_faculty = faculty
                        break
                if assigned_faculty and i != assigned_faculty:
                    problem += x[i, j, k, day, period] == 0


#objective function

### x[i,j,k,l] == 1 if faculty i teaches subject j to class k at period l else 0
 Example:\
   consider we have three periods,1,2,3 with 2 classes x,y and two subjects a and b\
   if subject a is assigned to period 1 and subject b is assigned to period 3 then:\
     **case 1**:objective = (1x1) + (0x2) + (1x3) = 4\
   the other case around is assigning subject a to period 1 and subject b to period b then:\
     **case 2**: objective = (1x1) + (1x2) + (0x3) = 3\
   both cases are feasible but the in second case the subjects are more evenly spread to the classes

In [12]:
# Objective Function: Prioritize faculty-subject assignments while minimizing gaps
objective = pulp.lpSum(x[i, j, k, day, period] * 5 for i in faculties for j in subjects for k in classes for day in sorted_days for period in periods_per_day) - pulp.lpSum(x[i, j, k, day, period] * (periods_per_day.index(period) + 1) for i in faculties for j in subjects for k in classes for day in sorted_days for period in periods_per_day)
problem += objective

In [13]:
# Solve the problem
problem.solve()

1

In [14]:
timetables = {}

In [15]:
import random

In [16]:
if pulp.LpStatus[problem.status] == "Optimal":
    # Extract assignment from the solution to generate the timetable
    timetable = {}

    for (i, j, k, day, period), var in x.items():
        if var.varValue == 1:
            if k not in timetable:
                timetable[k] = {day: {period: [] for period in periods_per_day} for day in sorted_days}
            timetable[k][day][period].append((i, j))

    # Save the weekly timetable for each class
    for class_name, class_data in timetable.items():
        class_timetable = {}
        for day in sorted_days:
            day_timetable = {}
            for period, assignments in class_data[day].items():
                if assignments:
                    day_timetable[period] = assignments
            if day_timetable:
                class_timetable[day] = day_timetable
        timetables[class_name] = class_timetable

else:
    print("No optimal solution found.")

In [17]:
timetables

{'ClassX': {'Monday': {1: [('FacultyB', 'Science')],
   2: [('FacultyB', 'Science'), ('FacultyD', 'Telugu')],
   3: [('FacultyB', 'Science'),
    ('FacultyC', 'History'),
    ('FacultyD', 'Telugu')],
   4: [('FacultyA', 'Math'), ('FacultyB', 'Science')],
   5: [('FacultyA', 'Math'), ('FacultyD', 'Telugu')]},
  'Tuesday': {1: [('FacultyB', 'Science'), ('FacultyD', 'Telugu')],
   2: [('FacultyA', 'Math')],
   3: [('FacultyA', 'Math'), ('FacultyB', 'Science')],
   4: [('FacultyC', 'History'), ('FacultyD', 'Telugu')],
   5: [('FacultyB', 'Science'),
    ('FacultyB', 'History'),
    ('FacultyC', 'Math')]},
  'Wednesday': {1: [('FacultyA', 'Math'),
    ('FacultyA', 'Science'),
    ('FacultyD', 'Telugu')],
   2: [('FacultyA', 'Science'), ('FacultyC', 'History')],
   3: [('FacultyC', 'History')],
   4: [('FacultyB', 'History'), ('FacultyD', 'Telugu')],
   5: [('FacultyA', 'Math'),
    ('FacultyB', 'Science'),
    ('FacultyC', 'History')]},
  'Thursday': {1: [('FacultyB', 'Science'),
    ('Facu

In [18]:
len(timetables['ClassX']['Monday'][1])

1

In [19]:
classes

['ClassX', 'ClassY']

In [21]:
timetables

{'ClassX': {'Monday': {1: [('FacultyB', 'Science')],
   2: [('FacultyB', 'Science'), ('FacultyD', 'Telugu')],
   3: [('FacultyB', 'Science'),
    ('FacultyC', 'History'),
    ('FacultyD', 'Telugu')],
   4: [('FacultyA', 'Math'), ('FacultyB', 'Science')],
   5: [('FacultyA', 'Math'), ('FacultyD', 'Telugu')]},
  'Tuesday': {1: [('FacultyB', 'Science'), ('FacultyD', 'Telugu')],
   2: [('FacultyA', 'Math')],
   3: [('FacultyA', 'Math'), ('FacultyB', 'Science')],
   4: [('FacultyC', 'History'), ('FacultyD', 'Telugu')],
   5: [('FacultyB', 'Science'),
    ('FacultyB', 'History'),
    ('FacultyC', 'Math')]},
  'Wednesday': {1: [('FacultyA', 'Math'),
    ('FacultyA', 'Science'),
    ('FacultyD', 'Telugu')],
   2: [('FacultyA', 'Science'), ('FacultyC', 'History')],
   3: [('FacultyC', 'History')],
   4: [('FacultyB', 'History'), ('FacultyD', 'Telugu')],
   5: [('FacultyA', 'Math'),
    ('FacultyB', 'Science'),
    ('FacultyC', 'History')]},
  'Thursday': {1: [('FacultyB', 'Science'),
    ('Facu